### Run notebook 02_LoL_predictor_Data_Preparation before running this one, as it prepares the data!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from tqdm import tqdm
import yaml

In [2]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

import wandb
wandb.login()

wandb.init(
        project="lol-predictor",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

wandb: Currently logged in as: mschon (lol-predictor). Use `wandb login --relogin` to force relogin


In [3]:
pd.set_option('display.max_columns', 500)
DATA_FOLDER = '../data/'https://wandb.ai/lol-predictor/lol-predictor

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv(f"{DATA_FOLDER}/train.csv")
test = pd.read_csv(f"{DATA_FOLDER}/test.csv")
validation = pd.read_csv(f"{DATA_FOLDER}/validation.csv")

In [5]:
train.shape

(6382, 21)

In [6]:
test.shape

(1482, 21)

In [7]:
validation.shape

(1482, 21)

In [8]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

In [9]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [10]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(config["preparation"]["features_amount"], nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [11]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [12]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [13]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

In [14]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [17]:
trainy = trainy.reshape((trainy.shape[0], 1))
testy = testy.reshape((testy.shape[0], 1))

In [18]:
model

Sequential(
  (0): Linear(in_features=20, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=32, bias=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): ReLU()
  (5): Linear(in_features=32, out_features=64, bias=True)
  (6): ReLU()
  (7): Linear(in_features=64, out_features=16, bias=True)
  (8): Dropout(p=0.5, inplace=False)
  (9): ReLU()
  (10): Linear(in_features=16, out_features=1, bias=True)
  (11): Sigmoid()
)

In [19]:
model.to(device)
print(device)

cpu


In [20]:
torch.cuda.is_available()

False

In [21]:
#torch.cuda.is_available()
#torch.cuda.get_device_name(0)

In [22]:
datasetTrain = torch.utils.data.TensorDataset(trainX, trainy)
loaderTrain = torch.utils.data.DataLoader(
    datasetTrain,
    batch_size=config['train']['batchSize'],
    shuffle=True
)

datasetTest = torch.utils.data.TensorDataset(testX, testy)
loaderTest = torch.utils.data.DataLoader(
    datasetTest,
    batch_size=config['train']['batchSize'],
    shuffle=True
)

In [23]:
for n in range(config['train']['epochs']):
    model.train()
    accuracy_sum = 0
    loss_sum = 0

    for (x,y) in tqdm(loaderTrain):
        y_pred = model(x)
        loss = compute_loss(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum = loss_sum + loss.cpu().detach().item()
        accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

    train_accuracy = accuracy_sum / len(loaderTrain)
    train_loss = loss_sum / len(loaderTrain)

    with torch.no_grad():
        accuracy_sum = 0
        loss_sum = 0

        for (x,y) in tqdm(loaderTest):
            y_pred = model(x)
            val_loss = compute_loss(y_pred, y)

            loss_sum = loss_sum + val_loss.cpu().detach().item()
            accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

    test_accuracy = accuracy_sum / len(loaderTest)
    train_loss = loss_sum / len(loaderTest)


    print(f'Epocha: {n}')
    print(f'Test accuracy: {test_accuracy} - Test Loss: {val_loss} |<->| Train Accuracy: {train_accuracy} - Train Loss: {loss}')
    wandb.log({'epoch': n, 'test_accuracy': test_accuracy, 'loss_val': val_loss, 'train_accuracy': train_accuracy, 'loss_train': loss})
print(loss)

100%|██████████████████████████████████████████| 47/47 [00:00<00:00, 852.81it/s]


Epocha: 0
Test accuracy: 70.4122340425532 - Test Loss: 0.3670816421508789 |<->| Train Accuracy: 61.841517857142854 - Train Loss: 0.5042333602905273


100%|█████████████████████████████████████████| 47/47 [00:00<00:00, 1466.56it/s]


Epocha: 1
Test accuracy: 72.84574468085107 - Test Loss: 0.5369664430618286 |<->| Train Accuracy: 70.50669642857143 - Train Loss: 0.5290387868881226


100%|█████████████████████████████████████████| 47/47 [00:00<00:00, 1139.70it/s]


...

Epocha: 47
Test accuracy: 72.89893617021276 - Test Loss: 0.8236886262893677 |<->| Train Accuracy: 72.30357142857143 - Train Loss: 0.46712496876716614


100%|█████████████████████████████████████████| 47/47 [00:00<00:00, 1367.24it/s]


Epocha: 48
Test accuracy: 72.77925531914893 - Test Loss: 0.5193194150924683 |<->| Train Accuracy: 72.97991071428572 - Train Loss: 0.2540875971317291


100%|█████████████████████████████████████████| 47/47 [00:00<00:00, 1053.63it/s]

Epocha: 49
Test accuracy: 72.52659574468085 - Test Loss: 0.539685070514679 |<->| Train Accuracy: 72.51785714285715 - Train Loss: 0.5275357365608215
tensor(0.5275, grad_fn=<BinaryCrossEntropyBackward0>)


In [24]:
fnashfujnafasf + asfbafhasnfaklol-predictor

NameError: name 'fnashfujnafasf' is not defined

## Tensorflow implementation

In [31]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

import tensorflow as tf
from tensorflow import keras

In [32]:
nHidden = config['train']['inputNeurons']

tf_model = keras.Sequential([
    keras.layers.Dense(units=20, activation='relu'),
    keras.layers.Dense(units=nHidden[0], activation='relu'),
    keras.layers.Dense(units=nHidden[1], activation='relu'),
    keras.layers.Dense(units=nHidden[2], activation='relu'),
    keras.layers.Dense(units=nHidden[3], activation='relu'),  #activation='softmax'
    keras.layers.Dense(units=1, activation='sigmoid')
])

tf_model.compile(optimizer='adam', 
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [33]:
trainX = train.drop(columns={"blueWins"})
trainy = train['blueWins']
testX = test.drop(columns={"blueWins"})
testy = test['blueWins']

In [ ]:
tf_model.fit(trainX, trainy, validation_data=(testX, testy), epochs=100, batch_size=32)

In [ ]:
tf_model.fit(trainX, trainy, validation_data=(testX, testy), epochs=50, batch_size=64)

Epoch 1/50
100/100 [==============================] - 0s 4ms/step - loss: 0.5491 - accuracy: 0.7195 - val_loss: 0.5566 - val_accuracy: 0.7287
Epoch 2/50
 27/100 [=======>......................] - ETA: 0s - loss: 0.5540 - accuracy: 0.7182

/home/xschon/.local/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


100/100 [==============================] - 0s 2ms/step - loss: 0.5482 - accuracy: 0.7181 - val_loss: 0.5460 - val_accuracy: 0.7301
Epoch 3/50
100/100 [==============================] - 0s 3ms/step - loss: 0.5514 - accuracy: 0.7200 - val_loss: 0.5482 - val_accuracy: 0.7274
Epoch 4/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5494 - accuracy: 0.7231 - val_loss: 0.5577 - val_accuracy: 0.7179
Epoch 5/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5443 - accuracy: 0.7242 - val_loss: 0.5528 - val_accuracy: 0.7287
Epoch 6/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5432 - accuracy: 0.7241 - val_loss: 0.5486 - val_accuracy: 0.7260
Epoch 7/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5445 - accuracy: 0.7223 - val_loss: 0.5466 - val_accuracy: 0.7227
Epoch 8/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5445 - accuracy: 0.7202 - val_loss: 0.5448 - val_accuracy: 0.7308
Epoch 9/50
100/10